In [1]:
import pandas as pd

In [2]:
filename = '/home/hb/machinelearning/Realtime-Fall-Detection-for-RNN/dataset/MobiFall_Dataset_v2.0/sub1/FALLS/SDL/SDL_ori_1_3.txt'

In [17]:
df =  pd.read_csv(filename, skiprows=16, header=None)

In [18]:
df.head()

,0,1,2,3
0,2263734564000,231.59665,8.406437,-65.78639
1,2263739503000,231.88106,10.277120,-68.14765
2,2263744482000,231.48561,8.862655,-65.99789
3,2263749482000,258.91132,11.825000,-66.32058
4,2263754447000,260.28268,12.254481,-66.41949
